### Анастасия Косятницына 

#### БКЛ-151

In [1]:
import string
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
import nltk
mystem = Mystem()
morph = MorphAnalyzer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Stoneberry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
%matplotlib inline  
pd.set_option('max_colwidth', 1000)

Импортируем данные:

In [3]:
train_data = pd.read_csv('data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

#### 1.0 Бейзлайн без нормализации

Без предворительной обработки данных посмотрим на accuracy

In [4]:
def vecrotizer(train_data, test_data):
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(train_data.values) 

    X_train = count_vectorizer.transform(train_data.values)
    X_test = count_vectorizer.transform(test_data.values)
    return X_train, X_test, count_vectorizer

In [5]:
X_train, X_test, count_vectorizer = vecrotizer(train_data.text, test_data.text)

In [6]:
X_train.shape

(8208, 20511)

In [7]:
X_test.shape

(2054, 20511)

In [8]:
y_train = train_data.label.values
y_test = test_data.label.values

In [9]:
from sklearn.model_selection import GridSearchCV

Перебор параметра C

In [10]:
def grid(X_train, y_train):
    log = LogisticRegression(penalty="l1")
    params = {'C': [0.01, 0.1, 1, 10, 100]}
    clf = GridSearchCV(log, param_grid=params)
    clf.fit(X_train, y_train)
    return clf

In [11]:
clf = grid(X_train, y_train)

In [12]:
clf.best_params_

{'C': 10}

Наилучший результат прогрмма показала при С = 10. Его и будем использовать в дальнейшем.

In [13]:
from sklearn.metrics import accuracy_score

def log(X_train2, y_train, X_test2, y_test):
    clf = LogisticRegression(penalty="l1", C=10)
    clf.fit(X_train2, y_train)
    y_pred2 = clf.predict(X_test2)
    normal = accuracy_score(y_test, y_pred2)
    return clf, normal, y_pred2

In [14]:
clf, base, y_pred = log(X_train, y_train, X_test, y_test)

Бейзлайн

In [15]:
base

0.6713729308666018

In [16]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       902
          0       0.66      0.75      0.71       972
          1       0.35      0.26      0.30       180

avg / total       0.67      0.67      0.67      2054

Макросредняя F1 мера -  0.5666229777804013
Микросредняя F1 мера -  0.6713729308666018


In [17]:
def print_important(vectorizer, clf, topn=10):
    features = vectorizer.get_feature_names()
    classes = clf.classes_
    importances = clf.coef_
    for i, cls in enumerate(classes):
        print('Значимые слова для класса - ', cls)
        important_words = sorted(list(zip(features, importances[i])), key=lambda x: abs(x[1]), reverse=True)[:topn]
        print([word for word,_ in important_words])
        print()

In [18]:
print_important(count_vectorizer, clf, topn=10)

Значимые слова для класса -  -1
['собираются', 'сбой', 'evakobb', 'обман', 'отправлялись', 'нему', 'керчи', 'перебоями', 'amaranth815', 'прогресс']

Значимые слова для класса -  0
['нему', 'собираются', 'хороший', 'жителям', 'прокомментировала', 'jivh4eenpq', 'расторгнуть', 'иа', 'прочность', 'гавно']

Значимые слова для класса -  1
['здорово', 'фотоед', 'специалистом', 'ожидал', 'радовать', 'обожаю', 't1cw25qbrz', 'расширяется', 'люблю', 'выросла']



#### 1.1 Добавить лемматизацию

Для улучшения работы программы уберем пунктуацию, стоп-слова из nltk.corpus и приведем слова к лемме.

In [19]:
stops = stopwords.words('russian')

In [20]:
from string import punctuation, digits

punctuation = set(punctuation + '«»—…“”\n\t' + digits)
punctuation.remove('@')
table = str.maketrans({ch: None for ch in punctuation})

In [21]:
def normalize(text):
    
    words_normalized_no_stops = []
    
    good_tokens = [word for word in word_tokenize(text) if len(text) > 1 
                                                    and not all([ch in punctuation for ch in word])]
    words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
    for word in words_normalized:
        if word not in stops:
            words_normalized_no_stops.append(word)
    
    return ' '.join(words_normalized_no_stops)

In [22]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

In [23]:
X_train2, X_test2, count_vectorizer = vecrotizer(train_data.normalized, test_data.normalized)

In [24]:
X_train2.shape

(8208, 14436)

In [25]:
X_test2.shape

(2054, 14436)

In [26]:
clf, normal, y_pred2 = log(X_train2, y_train, X_test2, y_test)

In [27]:
normal

0.6626095423563778

In [28]:
print(classification_report(y_test, y_pred2))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred2, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred2, average='micro'))

             precision    recall  f1-score   support

         -1       0.75      0.61      0.67       902
          0       0.65      0.77      0.71       972
          1       0.35      0.32      0.33       180

avg / total       0.67      0.66      0.66      2054

Макросредняя F1 мера -  0.5718232190860312
Микросредняя F1 мера -  0.6626095423563778


Результат ухудшился. Это может быть связано с тем, что стоп слова, которые используются по умолчанию включают в себя такие слова, которые могут повлиять на определение тональности твита. Например, не, которое в сочетании с глаголом, может кардинально поменять смысл всего сообщения (понравилось vs не понравилось). Попробуем посчитать без удаления стоп-слов.

#### 2. Проанализировать важные признаки, fp, fn, confusion matrix, изменить правила препроцессинга 

In [29]:
def normalize2(text):
    
    words_normalized_no_stops = []
    
    good_tokens = [word for word in word_tokenize(text) if len(text) > 1 
                                                    and not all([ch in punctuation for ch in word])]
    words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
    
    return ' '.join(words_normalized)

In [30]:
train_data['normalized_with_stops'] = train_data['text'].apply(normalize2)
test_data['normalized_with_stops'] = test_data['text'].apply(normalize2)

In [31]:
X_train3, X_test3, count_vectorizer = vecrotizer(train_data.normalized_with_stops, test_data.normalized_with_stops)

In [32]:
clf, normal_with_stops, y_pred3 = log(X_train3, y_train, X_test3, y_test)

In [33]:
normal_with_stops

0.6801363193768257

In [34]:
print(classification_report(y_test, y_pred3))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred3, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred3, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       902
          0       0.68      0.76      0.71       972
          1       0.42      0.33      0.37       180

avg / total       0.68      0.68      0.68      2054

Макросредняя F1 мера -  0.5945907742000814
Микросредняя F1 мера -  0.6801363193768257


Важные признаки:

In [35]:
print_important(count_vectorizer, clf, topn=10)

Значимые слова для класса -  -1
['задолженность', 'amaranth815', 'оштрафовать', 'атаковать', 'сбой', 'турбокнопка', 'уезжать', 'добиться', 'расторгнуть', 'испытывать']

Значимые слова для класса -  0
['650р', 'топливо', 'расторгнуть', 'гавный', 'испытывать', 'вносить', 'достоверно', 'задолженность', 'слогана', 'инноватор']

Значимые слова для класса -  1
['lizinastusha', 'здорово', 'адекватный', '6j3mfzcy5u', 'мило', 'топливо', 'расширяться', 'инноватор', 'youdicuudv', 'эфирный']



Достаточно низкий accuracy может быть объяснен тем, что, как видно из списка важных признаков, что некоторые слова являются важными сразу для нескольких классов. Например, испытывать, расторгнуть для -1 и 0; инноватор, топливо для 0 и 1.  Помимо этого, в список значимых признаков для отрицательного класса входят такие отрицательно маркированные слова, как атаковать, сбой, оштрафовать, задолженность, которые вряд могут встретиться в положительных твитах, что помогает хорошо отделить этот класс. Для нейтрального и положительного таких слов практически нет, кроме как здорово и мило у положительного. Это может повлиять на работу программы. Проверим свои догадки с помощью матрицы ошибок. 

Confusion matrix:

In [36]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred3)

array([[602, 269,  31],
       [186, 735,  51],
       [ 37,  83,  60]])

На основе результатов confusion_matrix видно, что у программы действительно возникают трудности в разграничивании нейтрального класса от положительного. 

#### 2.3 Подобрать параметры в классификаторе и векторайзере

In [37]:
?TfidfVectorizer

Сначала посмотрим на неизмененных данных:

In [38]:
def TfidfVectorize(train_data, test_data):
    tfidf = TfidfVectorizer()
    tfidf.fit(train_data.values)
    X_train = tfidf.transform(train_data.values)
    X_test = tfidf.transform(test_data.values)
    return X_train, X_test, tfidf

In [39]:
X_train, X_test, tfidf = TfidfVectorize(train_data.text, test_data.text)
clf, base_tf, y_pred = log(X_train, y_train, X_test, y_test)

base_tf

0.6635832521908471

In [40]:
print_important(tfidf, clf, topn=10)

Значимые слова для класса -  -1
['сбой', 'перебоями', 'собираются', 'tele2', 'нему', 'заблокировал', 'говно', 'траффику', 'подожгли', 'керчи']

Значимые слова для класса -  0
['иа', 'собираются', 'нему', 'жителям', 'прокомментировала', 'обратилась', 'вторая', 'вносил', 'просит', 'гавно']

Значимые слова для класса -  1
['люблю', 'ожидал', 'заработали', 'энергию', 'специалистом', 'текущих', 'выросла', 'обожаю', 'рубежом', 'заработал']



Нормализованные - стоп-слова

In [41]:
X_train2, X_test2, tfidf = TfidfVectorize(train_data.normalized, test_data.normalized)
clf, norm_tf, y_pred2 = log(X_train2, y_train, X_test2, y_test)

norm_tf 

0.6475170399221032

In [42]:
print_important(tfidf, clf, topn=10)

Значимые слова для класса -  -1
['задолженность', 'оштрафовать', 'tele2', 'сбой', 'позор', 'атаковать', 'говно', 'получаться', 'иа', 'угроза']

Значимые слова для класса -  0
['восстановление', 'иа', 'гавный', 'поведенческий', 'кончаться', 'топливо', 'позор', '650р', 'ловить', 'прокомментировать']

Значимые слова для класса -  1
['адекватный', 'понравиться', 'поезд', 'защита', 'частный', '6j3mfzcy5u', 'youdicuudv', 'мощь', 'топливо', 'инноватор']



Нормализованные + стоп-слова

In [43]:
X_train3, X_test3, tfidf = TfidfVectorize(train_data.normalized_with_stops, test_data.normalized_with_stops)
clf, norm_stops_tf, y_pred3 = log(X_train3, y_train, X_test3, y_test)

norm_stops_tf

0.6713729308666018

In [44]:
print_important(tfidf, clf, topn=10)

Значимые слова для класса -  -1
['оштрафовать', 'сбой', 'tele2', 'атаковать', 'говно', 'beeline_omsk', 'прекрасно', 'обман', 'уезжать', 'испытывать']

Значимые слова для класса -  0
['гавный', 'топливо', 'иа', 'восстановление', 'расторгнуть', '650р', 'поведенческий', 'доллар', 'хуй', 'инноватор']

Значимые слова для класса -  1
['lizinastusha', 'адекватный', 'здорово', 'понравиться', 'мощь', 'защита', 'инноватор', '6j3mfzcy5u', 'частный', 'довольный']



Итог

In [45]:
pd.DataFrame({'Data': ['Без изменений', 'Нормализация - стоп-слова', 'Нормализация + стоп-слова'],
              'CountV': [base, normal, normal_with_stops],
              'TfidfV': [base_tf, norm_tf, norm_stops_tf]
             })

,CountV,Data,TfidfV
0,0.671373,Без изменений,0.663583
1,0.662610,Нормализация - стоп-слова,0.647517
2,0.680136,Нормализация + стоп-слова,0.671373


Как видно из таблицы, Count_vectorizer показал себя лучше на наших данных.

Произведем отбор признаков:

Удалим ненужныe признаки, общие для всех классов

In [46]:
def unimportant(vectorizer, clf, topn=10):
    features = vectorizer.get_feature_names()
    classes = clf.classes_
    importances = clf.coef_
    answer = []
    for i, cls in enumerate(classes):
        # print('Не значимые слова для класса - ', cls)
        important_words = sorted(list(zip(features, importances[i])), key=lambda x: abs(x[1]), reverse=True)[len(importances) - topn:]
        answer.append(set(important_words))
        #print([word for word,_ in important_words])
        #print()
    return answer

In [75]:
unimp = unimportant(count_vectorizer, clf, topn=2500)

In [76]:
un_words = unimp[0] & unimp[1] & unimp[2]

In [77]:
stops = []
for word in un_words:
    stops.append(word[0])

In [78]:
len(stops)

1615

Удалим ненужные признаки

In [79]:
train_data['normalized_new_stops'] = train_data['text'].apply(normalize)
test_data['normalized_new_stops'] = test_data['text'].apply(normalize)


In [80]:
X_train4, X_test4, count_vectorizer = vecrotizer(train_data.normalized_new_stops,
                                                 test_data.normalized_new_stops)
clf, normalized_new_stops, y_pred4 = log(X_train4, y_train, X_test4, y_test)
normalized_new_stops

0.6815968841285297

In [81]:
print(classification_report(y_test, y_pred4))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred3, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred3, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       902
          0       0.68      0.76      0.72       972
          1       0.43      0.33      0.37       180

avg / total       0.68      0.68      0.68      2054

Макросредняя F1 мера -  0.5804887254021792
Микросредняя F1 мера -  0.6713729308666018


Нам удалось еще больше увеличить accuracy

In [ ]:
0.682570593962999

In [ ]:
2700